# Préparation des variables de mode de vie

In [ ]:
library(tidyverse)
library(factoextra)

# Charger les données
DataPilot <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/01_pilot_2025.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- DataPilot %>%
  select(all_of(variables_mode_de_vie))

In [1]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("lifestyle_typeTransport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))


Attachement du package : 'dplyr'


Les objets suivants sont masqu'es depuis 'package:stats':

    filter, lag


Les objets suivants sont masqu'es depuis 'package:base':

    intersect, setdiff, setequal, union




ERROR: Error: objet 'df_mode_de_vie' introuvable


In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

  0   1 
890 142 


Table pour la variable: ses_dwellingLoft 

   0    1 
1023    9 


Table pour la variable: ses_dwellingCondo 

  0   1 
935  97 


Table pour la variable: ses_dwellingTour 

  0   1 
977  55 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
495 537 


Table pour la variable: ses_dwellingTownhouse 

  0   1 
951  81 


Table pour la variable: ses_dwellingDuplex 

  0   1 
978  54 


Table pour la variable: ses_dwellingCoop 

   0    1 
1024    8 


Table pour la variable: ses_dwellingHLM 

   0    1 
1024    8 


Table pour la variable: ses_dwellingMobile 

   0    1 
1022   10 


Table pour la variable: ses_dwellingOther 

   0    1 
1001   31 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
429 603 


Table pour la variable: lifestyle_typeTransportSUV 

  0   1 
827 205 


Table pour la variable: lifestyle_typeTransportMoto 

   0    1 
1026    6 


Table pour la variable: lifestyle_typeTranspo

### Transformations

- Regroupement de ses_dwelling
  - ses_dwellingCondo : ses_dwellingCondo + ses_dwellingLoft + ses_dwellingTour
  - ses_dwellingOther : ses_dwellingOther + ses_dwellingCoop + ses_dwellingHLM + ses_dwellingMobile

- Regroupement des act_transport
  - lifestyle_typeTransportCar : lifestyle_typeTransportCar + lifestyle_typeTransportMoto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - lifestyle_typeTransportActive:   lifestyle_typeTransportWalk + lifestyle_typeTransportBicycle
  - Les autres ne sont pas regroupés

In [ ]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwellingCondo = if_else(
      ses_dwellingLoft == 1 | ses_dwellingTour == 1,
      1,
      ses_dwellingCondo
    ),
    ses_dwellingOther = if_else(
      ses_dwellingCoop == 1 | ses_dwellingHLM == 1 | ses_dwellingMobile == 1,
      1,
      ses_dwellingOther
    ),
    ses_dwellingDetachedHouse = if_else(
      ses_dwellingDetachedHouse == 1 | ses_dwellingTownhouse == 1,
      1,
      ses_dwellingDetachedHouse
    )
  ) %>%
  select(
    -ses_dwellingLoft,
    -ses_dwellingTour,
    -ses_dwellingCoop,
    -ses_dwellingHLM,
    -ses_dwellingMobile,
    -ses_dwellingTownhouse
  )

In [ ]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%
  mutate(
    lifestyle_typeTransportCar = if_else(
      lifestyle_typeTransportMoto == 1 | lifestyle_typeTransportSUV == 1,
      1,
      lifestyle_typeTransportCar
    ),
    lifestyle_typeTransportActive = if_else(
      lifestyle_typeTransportWalk == 1 | lifestyle_typeTransportBicycle == 1 | lifestyle_typeTransportPublicTransit == 1,
      1,
      0
    )
  ) %>%
  select(
    -lifestyle_typeTransportMoto,
    -lifestyle_typeTransportWalk,
    -lifestyle_typeTransportBicycle,
    -lifestyle_typeTransportSUV,
    -lifestyle_typeTransportPublicTransit)

In [ ]:
variables_mode_de_vie_clust <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwellingApp",
  "ses_dwellingCondo",
  "ses_dwellingDetachedHouse",
  "ses_dwellingDuplex",
  "ses_dwellingOther",
  "lifestyle_typeTransportCar",
  #"lifestyle_typeTransportSUV",
  "lifestyle_typeTransportActive"
  #"lifestyle_typeTransportPublicTransit"
)

In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

  0   1 
890 142 


Table pour la variable: ses_dwellingCondo 

  0   1 
871 161 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
414 618 


Table pour la variable: ses_dwellingDuplex 

  0   1 
978  54 


Table pour la variable: ses_dwellingOther 

  0   1 
975  57 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
218 814 


Table pour la variable: lifestyle_typeTransportActive 

  0   1 
818 214 




In [ ]:
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/02_pilot_2025_mode_de_vie.rds")

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.